In [1]:
import pandas as pd
import os

# Specify the folder path where your CSV files are stored
folder_path = './NSE'

# List of CSV file paths
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty DataFrame to store combined data
combined_data = pd.DataFrame()

# Loop through each CSV file and integrate data
for file in csv_files:
    # Extract company name from the file name (assuming format "companyname.csv")
    company_name = os.path.basename(file).split('.')[0]
    
    # Read the CSV file
    data = pd.read_csv(file)
    
    # Add Company_ID column using the extracted company name
    data['Company_ID'] = company_name
    
    # Concatenate to the combined DataFrame
    combined_data = pd.concat([combined_data, data], ignore_index=True)

combined_data.to_parquet('primary.parquet', index=False)

In [3]:
df = pd.read_parquet('primary.parquet')

# Display the first few rows
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Company_ID
0,2008-10-06,40.00,40.00,15.800,16.775,14.652864,23501600,20MICRONS
1,2008-10-07,16.00,19.00,13.925,15.025,13.124249,9113400,20MICRONS
2,2008-10-08,14.00,14.60,12.550,13.250,11.573798,2464384,20MICRONS
3,2008-10-10,12.45,12.45,10.825,11.600,10.132532,1207928,20MICRONS
4,2008-10-13,12.15,13.30,11.650,12.325,10.765814,898692,20MICRONS


In [2]:
import pandas as pd
import pandas_ta as ta

# Load the data from the parquet file
data = pd.read_parquet('primary.parquet')

# Ensure the Date column is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Get unique company IDs
company_ids = data['Company_ID'].unique()

# Function to handle missing values, data types, and prepare dataset
def preprocess_data(df):
    # Ensure Date column is sorted and remains as a column
    df = df.sort_values(by='Date')  # Ensure the data is ordered by Date
    
    # Ensure required columns are numeric and convert if needed
    required_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    for col in required_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Fill missing values with appropriate strategies
    df['Volume'] = df['Volume'].fillna(0)  # Handle missing volume data by filling with 0
    df.fillna(method='ffill', inplace=True)  # Forward fill other missing data
    return df

# Function to calculate technical indicators robustly
def calculate_indicators(df, company_id):
    df = preprocess_data(df)

    # List to track features that couldn't be engineered
    failed_features = []

    # Initialize empty columns for features in case of failure
    features = ['SMA_20', 'EMA_20', 'WMA_20', 'Upper_Band', 'Middle_Band', 'Lower_Band',
                'ATR_14', 'RSI_14', 'MACD', 'MACD_Signal', 'MACD_Hist', 'Stoch_K', 'Stoch_D',
                'OBV', 'VWAP', 'SAR', 'CCI_20', 'ROC']
    for feature in features:
        df[feature] = None

    try:
        # Check how many valid data points are in the 'Close' column
        valid_close_points = df['Close'].notna().sum()
        print(f"Processing {company_id} - Valid data points: {valid_close_points}")

        # Moving Averages
        try:
            if valid_close_points >= 20:
                df['SMA_20'] = ta.sma(df['Close'], length=20)
                df['EMA_20'] = ta.ema(df['Close'], length=20)
                df['WMA_20'] = ta.wma(df['Close'], length=20)
            else:
                raise ValueError(f"Not enough data for moving averages for {company_id}")
        except Exception as e:
            failed_features.extend(['SMA_20', 'EMA_20', 'WMA_20'])

        # Bollinger Bands
        try:
            if valid_close_points >= 20:
                bbands = ta.bbands(df['Close'], length=20)
                if bbands is not None:
                    df['Upper_Band'] = bbands['BBU_20_2.0']
                    df['Middle_Band'] = bbands['BBM_20_2.0']
                    df['Lower_Band'] = bbands['BBL_20_2.0']
                else:
                    raise ValueError("Bollinger Bands returned None")
            else:
                raise ValueError(f"Not enough data for Bollinger Bands for {company_id}")
        except Exception as e:
            failed_features.extend(['Upper_Band', 'Middle_Band', 'Lower_Band'])

        # ATR (Average True Range)
        try:
            if df[['High', 'Low', 'Close']].notna().sum().min() >= 14:
                df['ATR_14'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)
            else:
                raise ValueError(f"Not enough data for ATR for {company_id}")
        except Exception as e:
            failed_features.append('ATR_14')

        # RSI (Relative Strength Index)
        try:
            if valid_close_points >= 14:
                df['RSI_14'] = ta.rsi(df['Close'], length=14)
            else:
                raise ValueError(f"Not enough data for RSI for {company_id}")
        except Exception as e:
            failed_features.append('RSI_14')

        # MACD (Moving Average Convergence Divergence)
        try:
            if valid_close_points >= 26:
                fast_ema = ta.ema(df['Close'], length=12)
                slow_ema = ta.ema(df['Close'], length=26)
                if fast_ema.isnull().any() or slow_ema.isnull().any():
                    raise ValueError(f"Invalid EMA values for MACD for {company_id}")
                else:
                    macd = fast_ema - slow_ema
                    df['MACD_Signal'] = ta.ema(macd, length=9)
                    df['MACD_Hist'] = macd - df['MACD_Signal']
                    df['MACD'] = macd
            else:
                raise ValueError(f"Not enough data for MACD for {company_id}")
        except Exception as e:
            failed_features.extend(['MACD', 'MACD_Signal', 'MACD_Hist'])

        # Stochastic Oscillator
        try:
            if df[['High', 'Low', 'Close']].notna().sum().min() >= 14:
                stoch = ta.stoch(df['High'], df['Low'], df['Close'], k=14, d=3, smooth_k=3)
                if stoch is not None:
                    df['Stoch_K'], df['Stoch_D'] = stoch['STOCHk_14_3_3'], stoch['STOCHd_14_3_3']
                else:
                    raise ValueError(f"Stochastic Oscillator returned None for {company_id}")
            else:
                raise ValueError(f"Not enough data for Stochastic Oscillator for {company_id}")
        except Exception as e:
            failed_features.extend(['Stoch_K', 'Stoch_D'])

        # OBV (On-Balance Volume)
        try:
            if valid_close_points > 0 and df['Volume'].notna().sum() > 0:
                df['OBV'] = ta.obv(df['Close'], df['Volume'])
            else:
                raise ValueError(f"Not enough data for OBV for {company_id}")
        except Exception as e:
            failed_features.append('OBV')

        # VWAP (Volume Weighted Average Price)
        try:
            if df[['High', 'Low', 'Close', 'Volume']].notna().sum().min() > 0:
                df['VWAP'] = ta.vwap(df['High'], df['Low'], df['Close'], df['Volume'])
            else:
                raise ValueError(f"Not enough data for VWAP for {company_id}")
        except Exception as e:
            failed_features.append('VWAP')

        # SAR (Parabolic SAR)
        try:
            if df[['High', 'Low', 'Close']].notna().sum().min() >= 14:
                psar = ta.psar(df['High'], df['Low'], df['Close'])
                df['SAR'] = psar.iloc[:, 0] if psar is not None else None
            else:
                raise ValueError(f"Not enough data for SAR for {company_id}")
        except Exception as e:
            failed_features.append('SAR')

        # CCI (Commodity Channel Index)
        try:
            if df[['High', 'Low', 'Close']].notna().sum().min() >= 20:
                df['CCI_20'] = ta.cci(df['High'], df['Low'], df['Close'], length=20)
            else:
                raise ValueError(f"Not enough data for CCI for {company_id}")
        except Exception as e:
            failed_features.append('CCI_20')

        # Rate of Change (ROC)
        try:
            if valid_close_points >= 10:
                df['ROC'] = ta.roc(df['Close'], length=10)
            else:
                raise ValueError(f"Not enough data for ROC for {company_id}")
        except Exception as e:
            failed_features.append('ROC')

    except Exception as e:
        print(f"Error calculating indicators for {company_id}: {str(e)}")

    # Display the list of features that couldn't be engineered
    if failed_features:
        print(f"Features not engineered for {company_id}: {failed_features}")

    return df

# Initialize an empty DataFrame to store all results
all_data = pd.DataFrame()

# Process each company individually
for company_id in company_ids:
    # Filter data for the current company
    company_data = data[data['Company_ID'] == company_id]

    # Calculate indicators
    processed_data = calculate_indicators(company_data, company_id)

    # Append to the final DataFrame
    all_data = pd.concat([all_data, processed_data], ignore_index=True)

# Ensure 'Date' column remains as a column
if 'Date' in all_data.index.names:
    all_data.reset_index(inplace=True)

# Save the final DataFrame with all calculated features
all_data.to_parquet('processed_data_with_date.parquet')

Processing 20MICRONS - Valid data points: 3925
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for 20MICRONS: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing 21STCENMGM - Valid data points: 1643
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for 21STCENMGM: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing 360ONE - Valid data points: 1232
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for 360ONE: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing 3IINFOLTD - Valid data points: 4784
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWA

Features not engineered for ABMINTLLTD: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ABSLAMC - Valid data points: 721
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for ABSLAMC: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ACC - Valid data points: 5511
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for ACC: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ACCELYA - Valid data points: 5511
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for ACCELYA: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ACCURACY - Valid data points: 1521
[!] VWAP volume seri

Processing AGRITECH - Valid data points: 2617
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for AGRITECH: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing AGROPHOS - Valid data points: 1851
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for AGROPHOS: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing AGSTRA - Valid data points: 645
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for AGSTRA: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing AHL - Valid data points: 422
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series

Processing ALLCARGO - Valid data points: 4493
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for ALLCARGO: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ALLSEC - Valid data points: 4773
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for ALLSEC: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ALMONDZ - Valid data points: 4002
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for ALMONDZ: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ALOKINDS - Valid data points: 7204
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price s

Features not engineered for APCL: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing APCOTEXIND - Valid data points: 5480
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for APCOTEXIND: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing APEX - Valid data points: 1733
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for APEX: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing APLAPOLLO - Valid data points: 3142
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for APLAPOLLO: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing APLLTD - Valid data points: 3197
[!] VWAP volume

...

Features not engineered for UNIONBANK: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing UNIPARTS - Valid data points: 431
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for UNIPARTS: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing UNITDSPR - Valid data points: 71
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for UNITDSPR: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing UNITECH - Valid data points: 5514
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for UNITECH: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing UNITEDPOLY - Valid data points: 1848
[!] VWAP 

Features not engineered for VASWANI: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing VBL - Valid data points: 1938
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for VBL: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing VCL - Valid data points: 593
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for VCL: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing VEDL - Valid data points: 7209
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for VEDL: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing VENKEYS - Valid data points: 5511
[!] VWAP volume series is not datetime

Features not engineered for VISAKAIND: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing VISHNU - Valid data points: 2349
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for VISHNU: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing VISHWARAJ - Valid data points: 1216
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for VISHWARAJ: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing VIVIDHA - Valid data points: 2789
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for VIVIDHA: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing VLEGOV - Valid data points: 262
[!] VWAP volu

Features not engineered for WESTLIFE: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing WEWIN - Valid data points: 1522
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for WEWIN: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing WHEELS - Valid data points: 5511
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for WHEELS: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing WHIRLPOOL - Valid data points: 5483
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for WHIRLPOOL: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing WILLAMAGOR - Valid data points: 5512
[!] VWAP volu

Features not engineered for ZENITHSTL: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ZENSARTECH - Valid data points: 5511
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for ZENSARTECH: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ZENTEC - Valid data points: 2333
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for ZENTEC: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ZFCVINDIA - Valid data points: 5481
[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.
Features not engineered for ZFCVINDIA: ['MACD', 'MACD_Signal', 'MACD_Hist', 'VWAP']
Processing ZIMLAB - Valid data points: 441
[!] VWA

In [6]:
import pandas as pd
import pandas_ta as ta
from tqdm import tqdm
import os

# Load the original dataset
data = pd.read_parquet('primary.parquet')

# Ensure the Date column is in datetime format and is sorted
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data = data.dropna(subset=['Date'])  # Drop rows where Date is NaT
data = data.sort_values(by='Date')

# Get unique company IDs
company_ids = data['Company_ID'].unique()

# Function to handle missing values, data types, and ensure Date is properly ordered
def preprocess_data(df):
    # Ensure Date column is sorted and set as index
    df = df.sort_values(by='Date')
    df.set_index('Date', inplace=True)

    # Ensure required columns are numeric and convert if needed
    required_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    for col in required_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Fill missing values with forward fill (ffill) method
    df['Volume'] = df['Volume'].fillna(0)  # Handle missing volume data by filling with 0
    df.fillna(method='ffill', inplace=True)  # Forward fill other missing data
    return df

# Function to calculate VWAP and MACD-related features
def calculate_vwap_macd(df, company_id):
    df = preprocess_data(df)
    failed_features = []

    try:
        # VWAP (Volume Weighted Average Price)
        try:
            if df[['High', 'Low', 'Close', 'Volume']].notna().sum().min() > 0:
                df['VWAP'] = ta.vwap(df['High'], df['Low'], df['Close'], df['Volume'])
            else:
                raise ValueError(f"Not enough data for VWAP for {company_id}")
        except Exception as e:
            failed_features.append('VWAP')

        # MACD (Moving Average Convergence Divergence)
        try:
            if df['Close'].notna().sum() >= 26:
                macd_data = ta.macd(df['Close'])
                df['MACD'] = macd_data['MACD_12_26_9']
                df['MACD_Signal'] = macd_data['MACDs_12_26_9']
                df['MACD_Hist'] = macd_data['MACDh_12_26_9']
            else:
                raise ValueError(f"Not enough data for MACD for {company_id}")
        except Exception as e:
            failed_features.extend(['MACD', 'MACD_Signal', 'MACD_Hist'])

    except Exception as e:
        print(f"Error calculating VWAP or MACD for {company_id}: {str(e)}")

    # Display the list of features that couldn't be engineered
    if failed_features:
        print(f"Features not engineered for {company_id}: {failed_features}")

    return df

# Initialize an empty DataFrame to store all results
batch_size = 100
output_directory = 'batch_results'

# Create the directory for storing batches
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Process companies in batches
total_companies = len(company_ids)
batch_counter = 1

for i in tqdm(range(0, total_companies, batch_size), desc="Processing batches"):
    batch_ids = company_ids[i:i + batch_size]
    batch_data = pd.DataFrame()

    print(f"Processing batch {batch_counter} of companies...")

    for company_id in tqdm(batch_ids, desc=f"Batch {batch_counter} progress", leave=False):
        company_data = data[data['Company_ID'] == company_id]

        # Recalculate VWAP and MACD features
        processed_data = calculate_vwap_macd(company_data, company_id)

        # Append to the batch DataFrame
        batch_data = pd.concat([batch_data, processed_data], ignore_index=False)

    # Save the current batch to a separate file
    batch_file_path = os.path.join(output_directory, f'batch_{batch_counter}.parquet')
    batch_data.to_parquet(batch_file_path)

    print(f"Batch {batch_counter} processed and saved to {batch_file_path}.")
    batch_counter += 1

# After batch processing is done, combine all the batch files into one final file
final_data = pd.concat([pd.read_parquet(os.path.join(output_directory, f))
                        for f in os.listdir(output_directory) if f.endswith('.parquet')], ignore_index=False)

# Save the combined final dataset
final_data.to_parquet('final_vwap_macd_data_with_date.parquet')

print("Final dataset saved as 'final_vwap_macd_data_with_date.parquet'.")


Processing batches:   0%|                                                                       | 0/21 [00:00<?, ?it/s]

Processing batch 1 of companies...



Processing batches:   5%|███                                                            | 1/21 [00:41<13:57, 41.85s/it]

Batch 1 processed and saved to batch_results\batch_1.parquet.
Processing batch 2 of companies...



Processing batches:  10%|██████                                                         | 2/21 [01:23<13:09, 41.58s/it]

...


Batch 21 progress:  67%|███████████████████████████████████████████▎                     | 2/3 [00:01<00:00,  1.79it/s]

Features not engineered for STYLEBAAZA: ['MACD', 'MACD_Signal', 'MACD_Hist']



Processing batches: 100%|██████████████████████████████████████████████████████████████| 21/21 [15:22<00:00, 43.90s/it]

Features not engineered for GALAPREC: ['MACD', 'MACD_Signal', 'MACD_Hist']
Batch 21 processed and saved to batch_results\batch_21.parquet.


Final dataset saved as 'final_vwap_macd_data_with_date.parquet'.


In [8]:
# Load both datasets
dataset_1 = pd.read_parquet('final_vwap_macd_data_with_date.parquet')
dataset_2 = pd.read_parquet('processed_data_with_date.parquet')

# Function to check if 'Date' is in the columns or index, and fix if necessary
def ensure_date_column(df, dataset_name):
    if 'Date' not in df.columns:
        print(f"'Date' column not found in {dataset_name}. Checking if it's set as the index.")
        if 'Date' in df.index.names:
            print(f"'Date' is set as the index in {dataset_name}. Resetting index.")
            df.reset_index(inplace=True)  # Move 'Date' from index to column
        else:
            print(f"Error: 'Date' column not found in {dataset_name}.")
    return df

# Ensure 'Date' is present in both datasets as a column
dataset_1 = ensure_date_column(dataset_1, 'dataset_1')
dataset_2 = ensure_date_column(dataset_2, 'dataset_2')

# Ensure both 'Date' columns are in datetime format
dataset_1['Date'] = pd.to_datetime(dataset_1['Date'], errors='coerce')
dataset_2['Date'] = pd.to_datetime(dataset_2['Date'], errors='coerce')

# Ensure both 'Company_ID' columns are string types (to prevent merge issues)
dataset_1['Company_ID'] = dataset_1['Company_ID'].astype(str)
dataset_2['Company_ID'] = dataset_2['Company_ID'].astype(str)

# Merge both datasets on 'Date' and 'Company_ID' using an inner join
merged_data = pd.merge(dataset_1, dataset_2, on=['Date', 'Company_ID'], how='inner')

# Display the columns of the merged dataset to confirm the merge
print("Columns in merged dataset:")
print(merged_data.columns)

# Check for any missing values in the merged dataset
print("\nMissing values after merge:")
print(merged_data.isnull().sum())

# Save the merged dataset
merged_data.to_parquet('merged_final_dataset.parquet')

print("Merged dataset saved as 'merged_final_dataset.parquet'.")


'Date' column not found in dataset_1. Checking if it's set as the index.
'Date' is set as the index in dataset_1. Resetting index.
Columns in merged dataset:
Index(['Date', 'Open_x', 'High_x', 'Low_x', 'Close_x', 'Adj Close_x',
       'Volume_x', 'Company_ID', 'VWAP_x', 'MACD_x', 'MACD_Signal_x',
       'MACD_Hist_x', 'Open_y', 'High_y', 'Low_y', 'Close_y', 'Adj Close_y',
       'Volume_y', 'SMA_20', 'EMA_20', 'WMA_20', 'Upper_Band', 'Middle_Band',
       'Lower_Band', 'ATR_14', 'RSI_14', 'MACD_y', 'MACD_Signal_y',
       'MACD_Hist_y', 'Stoch_K', 'Stoch_D', 'OBV', 'VWAP_y', 'SAR', 'CCI_20',
       'ROC'],
      dtype='object')

Missing values after merge:
Date                   0
Open_x                 0
High_x                 0
Low_x                  0
Close_x                0
Adj Close_x            0
Volume_x               0
Company_ID             0
VWAP_x            258976
MACD_x             49865
MACD_Signal_x      65689
MACD_Hist_x        65689
Open_y                 0
High_y    

In [ ]:
print(merged_data.isnull().sum())

In [4]:
import pandas as pd
# Load the merged dataset
merged_data = pd.read_parquet('merged_final_dataset.parquet')

# Drop the columns you want to remove
columns_to_remove = ['Open_y', 'High_y', 'Low_y', 'Close_y', 'Adj Close_y', 'Volume_y', 'MACD_y', 'MACD_Signal_y', 'MACD_Hist_y', 'VWAP_y' ]
merged_data = merged_data.drop(columns=columns_to_remove)

# Verify the columns have been removed
print("Columns in the dataset after removing unwanted columns:")
print(merged_data.columns)

# Save the cleaned dataset
merged_data.to_parquet('cleaned_merged_final_dataset.parquet')

print("Cleaned dataset saved as 'intermediate_technical_features.parquet'.")


Columns in the dataset after removing unwanted columns:
Index(['Date', 'Open_x', 'High_x', 'Low_x', 'Close_x', 'Adj Close_x',
       'Volume_x', 'Company_ID', 'VWAP_x', 'MACD_x', 'MACD_Signal_x',
       'MACD_Hist_x', 'SMA_20', 'EMA_20', 'WMA_20', 'Upper_Band',
       'Middle_Band', 'Lower_Band', 'ATR_14', 'RSI_14', 'Stoch_K', 'Stoch_D',
       'OBV', 'SAR', 'CCI_20', 'ROC'],
      dtype='object')
Cleaned dataset saved as 'intermediate_technical_features.parquet'.


In [5]:
print(merged_data.isnull().sum())

Date                   0
Open_x                 0
High_x                 0
Low_x                  0
Close_x                0
Adj Close_x            0
Volume_x               0
Company_ID             0
VWAP_x            258976
MACD_x             49865
MACD_Signal_x      65689
MACD_Hist_x        65689
SMA_20             37935
EMA_20             37935
WMA_20             37935
Upper_Band         37935
Middle_Band        37935
Lower_Band         37935
ATR_14             27980
RSI_14             32665
Stoch_K            29976
Stoch_D            33956
OBV                    0
SAR              3289456
CCI_20            114938
ROC                20003
dtype: int64


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Load dataset from Parquet
input_file = 'cleaned_merged_final_dataset.parquet'  # Replace with your actual file path
data = pd.read_parquet(input_file)

# Ensure 'Date' is in the correct format (if needed, depending on the original dataset's Date format)
data['Date'] = pd.to_datetime(data['Date'])

# Sort data by Date and Company_ID to ensure consistency
data = data.sort_values(by=['Company_ID', 'Date']).reset_index(drop=True)

# List of unique companies
companies = data['Company_ID'].unique()

# Placeholder for the final result after processing each company in batches
final_data = pd.DataFrame()

# --- Batch processing for each company ---
for company in tqdm(companies, desc='Processing Companies', unit='company'):
    company_data = data[data['Company_ID'] == company].copy()  # Create a copy to avoid changing the original

    # --- Derived Features from Existing Columns ---

    # 1. Price Change and Volume Change
    company_data['Price_Change'] = company_data['Close_x'] - company_data['Open_x']
    company_data['Price_Change_Percent'] = (company_data['Close_x'] - company_data['Close_x'].shift(1)) / company_data['Close_x'].shift(1) * 100
    company_data['Volume_Change'] = company_data['Volume_x'] - company_data['Volume_x'].shift(1)
    company_data['Volume_Change_Percent'] = (company_data['Volume_x'] - company_data['Volume_x'].shift(1)) / company_data['Volume_x'].shift(1) * 100

    # 2. High-Low Spread
    company_data['High_Low_Spread'] = company_data['High_x'] - company_data['Low_x']

    # 3. Price Ratios
    company_data['Close_Open_Ratio'] = company_data['Close_x'] / company_data['Open_x']
    company_data['High_Low_Ratio'] = company_data['High_x'] / company_data['Low_x']
    company_data['Price_Range'] = (company_data['Close_x'] - company_data['Low_x']) / (company_data['High_x'] - company_data['Low_x'])
    company_data['Price_Volume_Product'] = company_data['Close_x'] * company_data['Volume_x']

    # 4. Lagged Prices and Volume (for 1, 5, 10 days)
    lags = [1, 5, 10]
    for lag in lags:
        company_data[f'Lag_Close_x_{lag}'] = company_data['Close_x'].shift(lag)
        company_data[f'Lag_Volume_x_{lag}'] = company_data['Volume_x'].shift(lag)

    # --- Time-Based Features ---
    company_data['Day_of_Week'] = company_data['Date'].dt.dayofweek
    company_data['Month'] = company_data['Date'].dt.month
    company_data['Quarter'] = company_data['Date'].dt.quarter
    company_data['Week_of_Year'] = company_data['Date'].dt.isocalendar().week

    # --- Volatility and Trend Features ---
    rolling_window = 20  # Define a standard rolling window

    # Rolling standard deviation (volatility) for Close and Volume
    company_data['Rolling_Std_Close'] = company_data['Close_x'].rolling(window=rolling_window).std()
    company_data['Rolling_Std_Volume'] = company_data['Volume_x'].rolling(window=rolling_window).std()

    # Rolling mean (Moving Average)
    company_data['Rolling_Mean_Close'] = company_data['Close_x'].rolling(window=rolling_window).mean()

    # Rolling Max/Min
    company_data['Rolling_Max_Close'] = company_data['Close_x'].rolling(window=rolling_window).max()
    company_data['Rolling_Min_Close'] = company_data['Close_x'].rolling(window=rolling_window).min()

    # Rolling price range
    company_data['Rolling_Price_Range'] = company_data['Rolling_Max_Close'] - company_data['Rolling_Min_Close']

    # Z-Score Normalization
    company_data['Z_Score_Close'] = (company_data['Close_x'] - company_data['Rolling_Mean_Close']) / company_data['Rolling_Std_Close']

    # --- Momentum and Trend Features ---
    momentum_window = 5  # For 5-day momentum

    # Momentum (Close_x - Close_x_n)
    company_data['Momentum'] = company_data['Close_x'].diff(momentum_window)

    # Price Acceleration
    company_data['Price_Acceleration'] = company_data['Price_Change'] - company_data['Price_Change'].shift(1)

    # --- Liquidity and Volume Features ---
    # OBV Change (already included in OBV column)
    company_data['OBV_Change'] = company_data['OBV'] - company_data['OBV'].shift(1)

    # Volume Ratio
    company_data['Volume_Ratio'] = company_data['Volume_x'] / company_data['Volume_x'].rolling(window=rolling_window).mean()

    # --- Statistical and Risk-Based Features ---
    # Rolling returns (percentage change)
    company_data['Rolling_Return'] = company_data['Close_x'].pct_change().rolling(window=rolling_window).mean()

    # Kurtosis and Skewness
    company_data['Kurtosis'] = company_data['Close_x'].rolling(window=rolling_window).kurt()
    company_data['Skewness'] = company_data['Close_x'].rolling(window=rolling_window).skew()

    # --- Cleaning: Fill missing data appropriately ---
    company_data.fillna(method='ffill', inplace=True)
    company_data.fillna(method='bfill', inplace=True)

    # Append the processed company data to the final DataFrame
    final_data = pd.concat([final_data, company_data], axis=0)

# Save the final processed dataset to a new parquet file
output_file = 'intermediate_self_sufficient.parquet'
final_data.to_parquet(output_file, index=False)

# Output confirmation
print(f"Processed data saved to {output_file}")


Processing Companies: 100%|███████████████████████████████████████████████████| 2003/2003 [29:53<00:00,  1.12company/s]


Processed data saved to intermediate_self_sufficient.parquet


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Load dataset from the parquet file
input_file = 'intermediate_self_sufficient.parquet'  # Replace with the correct file path
data = pd.read_parquet(input_file)

# Ensure 'Date' is in the correct format (keeping it intact)
data['Date'] = pd.to_datetime(data['Date'])

# Sort data by Date and Company_ID to ensure consistency
data = data.sort_values(by=['Company_ID', 'Date']).reset_index(drop=True)

# List of unique companies for batch processing
companies = data['Company_ID'].unique()

# Placeholder for the final result after processing each company in batches
final_data = pd.DataFrame()

# --- Batch processing for each company ---
for company in tqdm(companies, desc='Processing Companies', unit='company'):
    company_data = data[data['Company_ID'] == company].copy()  # Create a copy of each company's data

    # --- Sharpe Ratio ---
    risk_free_rate = 0  # Setting the risk-free rate to 0 for simplicity
    company_data['Sharpe_Ratio'] = (company_data['Rolling_Return'] - risk_free_rate) / company_data['Rolling_Std_Close']

    # --- Fibonacci Retracement Levels ---
    fibonacci_levels = [0.236, 0.382, 0.618]
    for level in fibonacci_levels:
        company_data[f'Fib_Level_{level}'] = company_data['High_x'] - (company_data['High_x'] - company_data['Low_x']) * level

    # --- Pivot Points ---
    company_data['Pivot_Point'] = (company_data['High_x'] + company_data['Low_x'] + company_data['Close_x']) / 3

    # --- Additional ROC Calculations ---
    company_data['ROC_5'] = (company_data['Close_x'] - company_data['Close_x'].shift(5)) / company_data['Close_x'].shift(5) * 100
    company_data['ROC_10'] = (company_data['Close_x'] - company_data['Close_x'].shift(10)) / company_data['Close_x'].shift(10) * 100

    # --- Cleaning: Fill missing data appropriately ---
    company_data.fillna(method='ffill', inplace=True)
    company_data.fillna(method='bfill', inplace=True)

    # Append the processed company data to the final DataFrame
    final_data = pd.concat([final_data, company_data], axis=0)

# Save the final processed dataset to a new parquet file
output_file = 'intermediate_self_sufficient_2.parquet'
final_data.to_parquet(output_file, index=False)

# Output confirmation
print(f"Processed data saved to {output_file}")


Processing Companies: 100%|███████████████████████████████████████████████████| 2003/2003 [33:42<00:00,  1.01s/company]


Processed data saved to intermediate_self_sufficient_2.parquet


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm

# Load the dataset (replace with actual file path)
print("Loading dataset...")
df = pd.read_parquet('merged_with_weighted_sentiment_final.parquet')
print("Dataset loaded.")

# Ensure 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Define the market and global index symbols
market_index_symbol = '^NSEI'  # Nifty 50
global_index_symbol = '^GSPC'  # S&P 500 (as an example of a global index)

# Step 3: Fetch Market Index Data
print("Fetching market index data (Nifty 50)...")
market_index = yf.download(market_index_symbol, start=df['Date'].min(), end=df['Date'].max())
market_index = market_index[['Adj Close']].rename(columns={'Adj Close': 'Market_Close'})
market_index['Market_Returns'] = market_index['Market_Close'].pct_change()

print("Fetching global index data (S&P 500)...")
global_index = yf.download(global_index_symbol, start=df['Date'].min(), end=df['Date'].max())
global_index = global_index[['Adj Close']].rename(columns={'Adj Close': 'Global_Close'})
global_index['Global_Returns'] = global_index['Global_Close'].pct_change()

# Step 4: Merge Market and Global Index Data with Stock Data
print("Merging market and global index data with stock data...")
df = df.merge(market_index, how='left', left_on='Date', right_index=True)
df = df.merge(global_index, how='left', left_on='Date', right_index=True)
print("Merge completed.")

# Function to calculate rolling Beta (using a rolling window)
def rolling_beta(stock_returns, market_returns, window):
    cov_matrix = stock_returns.rolling(window=window).cov(market_returns)
    var_market = market_returns.rolling(window=window).var()
    rolling_beta = cov_matrix / var_market
    return rolling_beta

# Step 5: Process in Batches
window = 60  # 60-day rolling window (adjust as necessary)
batch_size = 50  # Adjust this based on your system's memory capacity

# Initialize columns for rolling metrics
df['Rolling_Beta'] = np.nan
df['Rolling_Market_Correlation'] = np.nan
df['Rolling_Global_Correlation'] = np.nan

# Get the list of unique companies
unique_companies = df['Company_ID'].unique()

# Process the data in batches to avoid memory issues
num_batches = len(unique_companies) // batch_size + 1

for batch_idx in tqdm(range(num_batches), desc="Processing batches"):
    # Get the companies in the current batch
    batch_companies = unique_companies[batch_idx * batch_size : (batch_idx + 1) * batch_size]

    # Filter data for the batch of companies
    batch_data = df[df['Company_ID'].isin(batch_companies)]

    for company_id in tqdm(batch_companies, desc="Processing companies in batch", leave=False):
        # Filter data for the company
        company_data = batch_data[batch_data['Company_ID'] == company_id].copy()

        # Calculate company returns
        company_data['Company_Returns'] = company_data['Close_x'].pct_change()

        # Drop NaN values for calculations
        valid_data = company_data.dropna(subset=['Company_Returns', 'Market_Returns', 'Global_Returns'])

        if not valid_data.empty:
            # Calculate rolling Beta with the market index
            df.loc[valid_data.index, 'Rolling_Beta'] = rolling_beta(valid_data['Company_Returns'], valid_data['Market_Returns'], window)

            # Calculate rolling correlation with the market index
            df.loc[valid_data.index, 'Rolling_Market_Correlation'] = valid_data['Company_Returns'].rolling(window).corr(valid_data['Market_Returns'])

            # Calculate rolling correlation with the global index
            df.loc[valid_data.index, 'Rolling_Global_Correlation'] = valid_data['Company_Returns'].rolling(window).corr(valid_data['Global_Returns'])

    # Save intermediate results to a file after each batch (optional but recommended)
    df.to_parquet(f'intermediate_batch_{batch_idx}.parquet')
    print(f"Batch {batch_idx} processed and saved.")

# Step 6: Save the final DataFrame with all rolling features
print("Saving the final dataset...")
df.to_parquet('final_with_company_specific_rolling_features.parquet')
print("Company-specific rolling features saved in 'final_with_company_specific_rolling_features.parquet'.")


Loading dataset...
Dataset loaded.
Fetching market index data (Nifty 50)...


[*********************100%***********************]  1 of 1 completed


Fetching global index data (S&P 500)...


[*********************100%***********************]  1 of 1 completed


Merging market and global index data with stock data...
Merge completed.


...


Processing batches:  98%|████████████████████████████████████████████████████████████▍ | 39/40 [32:11<00:50, 50.46s/it]

Batch 38 processed and saved.



Processing batches: 100%|██████████████████████████████████████████████████████████████| 40/40 [33:06<00:00, 49.67s/it]

Batch 39 processed and saved.


Saving the final dataset...
Company-specific rolling features saved in 'final_with_company_specific_rolling_features.parquet'.


In [2]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import requests

# Load the dataset (replace with actual file path)
print("Loading dataset...")
df = pd.read_parquet('final_with_company_specific_rolling_features.parquet')
print("Dataset loaded.")

# Ensure 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Function to fetch Yahoo Finance data
def fetch_yahoo_data(symbol, feature_name):
    print(f"Fetching {feature_name} data...")
    try:
        data = yf.download(symbol, start=df['Date'].min(), end=df['Date'].max())
        if not data.empty:
            data = data[['Adj Close']].rename(columns={'Adj Close': feature_name})
            data['Date'] = data.index
            data.reset_index(drop=True, inplace=True)  # Reset index to avoid conflicts
            return data[['Date', feature_name]]
        else:
            print(f"No data found for {symbol}")
            return pd.DataFrame(columns=['Date', feature_name])
    except Exception as e:
        print(f"Failed to fetch {feature_name} data for {symbol}: {e}")
        return pd.DataFrame(columns=['Date', feature_name])

# Fetch INR/USD Exchange Rate (using Yahoo Finance)
currency_data = fetch_yahoo_data('INR=X', 'INR_USD_Close')

# Fetch Crude Oil Prices (using Yahoo Finance)
crude_oil_data = fetch_yahoo_data('CL=F', 'Crude_Oil_Close')

# Fetch Gold Prices (using Yahoo Finance)
gold_data = fetch_yahoo_data('GC=F', 'Gold_Close')

# Fetch India VIX (Volatility Index) using Yahoo Finance (fallback if unavailable)
vix_data = fetch_yahoo_data('^INDIAVIX', 'India_VIX')
if vix_data.empty:
    print("India VIX data is not available on Yahoo Finance. Please check alternative sources or data periods.")

# Function to fetch World Bank data
def fetch_world_bank_data(indicator, feature_name):
    print(f"Fetching {feature_name} data from World Bank...")
    try:
        url = f'http://api.worldbank.org/v2/country/IN/indicator/{indicator}?format=json&date=1990:2024'
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors
        data = pd.DataFrame(response.json()[1])  # The second part of the response is the data
        data['Date'] = pd.to_datetime(data['date'])
        data = data.rename(columns={'value': feature_name})
        return data[['Date', feature_name]]
    except (requests.exceptions.HTTPError, IndexError, KeyError) as e:
        print(f"Failed to fetch {feature_name} from World Bank API: {e}")
        return pd.DataFrame(columns=['Date', feature_name])

# Fetch CPI Inflation and Interest Rates from World Bank
inflation_data = fetch_world_bank_data('FP.CPI.TOTL', 'CPI_Inflation')  # CPI Inflation Indicator
interest_rate_data = fetch_world_bank_data('FR.INR.LEND', 'Interest_Rate')  # Interest Rate Indicator

# Merge external data into the main DataFrame
print("Merging external data with the main dataset...")

# Ensure 'Date' is a column and not part of the index before merging
df.reset_index(drop=True, inplace=True)

# Merge INR/USD exchange rate
if not currency_data.empty:
    df = df.merge(currency_data, on='Date', how='left')

# Merge Crude Oil prices
if not crude_oil_data.empty:
    df = df.merge(crude_oil_data, on='Date', how='left')

# Merge Gold prices
if not gold_data.empty:
    df = df.merge(gold_data, on='Date', how='left')

# Merge India VIX data
if not vix_data.empty:
    df = df.merge(vix_data, on='Date', how='left')

# Merge Inflation (CPI) data
if not inflation_data.empty:
    df = df.merge(inflation_data, on='Date', how='left')

# Merge Interest Rate data
if not interest_rate_data.empty:
    df = df.merge(interest_rate_data, on='Date', how='left')

# Fill missing values (for days without data) with forward fill
df.fillna(method='ffill', inplace=True)

# Save the final dataset with external features
df.to_parquet('final_with_external_features_and_rates.parquet')
print("Final dataset with external features saved as 'final_with_external_features_and_rates.parquet'.")


Loading dataset...
Dataset loaded.
Fetching INR_USD_Close data...


[*********************100%***********************]  1 of 1 completed


Fetching Crude_Oil_Close data...


[*********************100%***********************]  1 of 1 completed


Fetching Gold_Close data...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^INDIAVIX']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 1991-01-16 00:00:00 -> 2024-09-11 00:00:00)')


Fetching India_VIX data...
No data found for ^INDIAVIX
India VIX data is not available on Yahoo Finance. Please check alternative sources or data periods.
Fetching CPI_Inflation data from World Bank...
Failed to fetch CPI_Inflation from World Bank API: 502 Server Error: Bad Gateway for url: http://api.worldbank.org/v2/country/IN/indicator/FP.CPI.TOTL?format=json&date=1990:2024
Fetching Interest_Rate data from World Bank...
Failed to fetch Interest_Rate from World Bank API: 502 Server Error: Bad Gateway for url: http://api.worldbank.org/v2/country/IN/indicator/FR.INR.LEND?format=json&date=1990:2024
Merging external data with the main dataset...
Final dataset with external features saved as 'final_with_external_features_and_rates.parquet'.


In [ ]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# Load a small chunk of dataset for testing
print("Loading a small chunk of dataset for testing...")
df = pd.read_parquet('final_with_external_features_and_rates.parquet')  # Replace with the correct file path
df = df.head(1000)  # Load only a small chunk for testing
print("Dataset chunk loaded.")
print("Initial columns:", df.columns.tolist())

# Ensure 'Date' column is in datetime format in the main dataset
df['Date'] = pd.to_datetime(df['Date'])

# Function to fetch Yahoo Finance data
def fetch_yahoo_data(symbol, feature_name):
    print(f"Fetching {feature_name} data for {symbol}...")
    try:
        data = yf.download(symbol, start=df['Date'].min(), end=df['Date'].max())
        if not data.empty:
            data = data[['Adj Close']].rename(columns={'Adj Close': feature_name})
            data['Date'] = data.index
            data.reset_index(drop=True, inplace=True)
            data['Date'] = pd.to_datetime(data['Date'])
            print(f"Fetched {len(data)} records for {feature_name}.")
            return data[['Date', feature_name]]
        else:
            print(f"No data found for {symbol}")
            return pd.DataFrame(columns=['Date', feature_name])
    except Exception as e:
        print(f"Failed to fetch {feature_name} data for {symbol}: {e}")
        return pd.DataFrame(columns=['Date', feature_name])

# Function to merge data in batches
def merge_in_batches(df, data_to_merge, feature_name):
    if data_to_merge.empty:
        print(f"Skipping {feature_name} - no data available to merge.")
        return df
    print(f"Merging {feature_name} with the main dataset...")
    df = df.merge(data_to_merge, on='Date', how='left')
    return df

# Fetch external data for testing
print("Fetching external data for testing...")
vix_data = fetch_yahoo_data('^VIX', 'VIX_Close')
us_inflation_data = fetch_yahoo_data('CPIAUCNS', 'US_CPI_Inflation')
us_interest_rate_data = fetch_yahoo_data('DFF', 'US_Federal_Funds_Rate')

# Check if date ranges overlap
print("Main dataset date range:", df['Date'].min(), "to", df['Date'].max())
print("VIX_Close date range:", vix_data['Date'].min(), "to", vix_data['Date'].max())
print("US_CPI_Inflation date range:", us_inflation_data['Date'].min(), "to", us_inflation_data['Date'].max())
print("US_Federal_Funds_Rate date range:", us_interest_rate_data['Date'].min(), "to", us_interest_rate_data['Date'].max())

# Merging VIX_Close
df = merge_in_batches(df, vix_data, 'VIX_Close')
print("Columns after merging VIX_Close:", df.columns.tolist())

# Merging US CPI Inflation
df = merge_in_batches(df, us_inflation_data, 'US_CPI_Inflation')
print("Columns after merging US_CPI_Inflation:", df.columns.tolist())

# Merging US Federal Funds Rate
df = merge_in_batches(df, us_interest_rate_data, 'US_Federal_Funds_Rate')
print("Columns after merging US_Federal_Funds_Rate:", df.columns.tolist())

# Check missing values and print a preview
print("\nMissing values after merging:\n", df.isnull().sum())
print("\nPreview of merged data:\n", df.head())

# Proceed or stop based on user input
proceed = input("Do you want to continue with the full dataset? (yes/no): ")
if proceed.lower() == 'yes':
    print("Loading full dataset...")
    full_df = pd.read_parquet('final_with_external_features_and_rates.parquet')
    full_df['Date'] = pd.to_datetime(full_df['Date'])

    # Merging the full dataset with VIX data
    full_df = merge_in_batches(full_df, vix_data, 'VIX_Close')

    # Merging the full dataset with US CPI Inflation data
    full_df = merge_in_batches(full_df, us_inflation_data, 'US_CPI_Inflation')

    # Merging the full dataset with US Federal Funds Rate data
    full_df = merge_in_batches(full_df, us_interest_rate_data, 'US_Federal_Funds_Rate')

    # Save the final dataset with external features
    full_df.to_parquet('final_with_external_features_and_rates_and_inflation_full.parquet')
    print("Final dataset with external features saved as 'final_with_external_features_and_rates_and_inflation_full.parquet'.")
else:
    print("Execution stopped as per user request.")


Loading a small chunk of dataset for testing...
Dataset chunk loaded.
Initial columns: ['Date', 'Open_x', 'High_x', 'Low_x', 'Close_x', 'Adj Close_x', 'Volume_x', 'Company_ID', 'VWAP_x', 'MACD_x', 'MACD_Signal_x', 'MACD_Hist_x', 'SMA_20', 'EMA_20', 'WMA_20', 'Upper_Band', 'Middle_Band', 'Lower_Band', 'ATR_14', 'RSI_14', 'Stoch_K', 'Stoch_D', 'OBV', 'SAR', 'CCI_20', 'ROC', 'Price_Change', 'Price_Change_Percent', 'Volume_Change', 'Volume_Change_Percent', 'High_Low_Spread', 'Close_Open_Ratio', 'High_Low_Ratio', 'Price_Range', 'Price_Volume_Product', 'Lag_Close_x_1', 'Lag_Volume_x_1', 'Lag_Close_x_5', 'Lag_Volume_x_5', 'Lag_Close_x_10', 'Lag_Volume_x_10', 'Day_of_Week', 'Month', 'Quarter', 'Week_of_Year', 'Rolling_Std_Close', 'Rolling_Std_Volume', 'Rolling_Mean_Close', 'Rolling_Max_Close', 'Rolling_Min_Close', 'Rolling_Price_Range', 'Z_Score_Close', 'Momentum', 'Price_Acceleration', 'OBV_Change', 'Volume_Ratio', 'Rolling_Return', 'Kurtosis', 'Skewness', 'Sharpe_Ratio', 'Fib_Level_0.236', '

[*********************100%***********************]  1 of 1 completed


Fetched 1022 records for VIX_Close.
Fetching US_CPI_Inflation data for CPIAUCNS...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['CPIAUCNS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['DFF']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2008-10-21 00:00:00 -> 2012-11-12 00:00:00)')


No data found for CPIAUCNS
Fetching US_Federal_Funds_Rate data for DFF...
No data found for DFF
Main dataset date range: 2008-10-21 00:00:00 to 2012-11-12 00:00:00
VIX_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
US_CPI_Inflation date range: nan to nan
US_Federal_Funds_Rate date range: nan to nan
Merging VIX_Close with the main dataset...
Columns after merging VIX_Close: ['Date', 'Open_x', 'High_x', 'Low_x', 'Close_x', 'Adj Close_x', 'Volume_x', 'Company_ID', 'VWAP_x', 'MACD_x', 'MACD_Signal_x', 'MACD_Hist_x', 'SMA_20', 'EMA_20', 'WMA_20', 'Upper_Band', 'Middle_Band', 'Lower_Band', 'ATR_14', 'RSI_14', 'Stoch_K', 'Stoch_D', 'OBV', 'SAR', 'CCI_20', 'ROC', 'Price_Change', 'Price_Change_Percent', 'Volume_Change', 'Volume_Change_Percent', 'High_Low_Spread', 'Close_Open_Ratio', 'High_Low_Ratio', 'Price_Range', 'Price_Volume_Product', 'Lag_Close_x_1', 'Lag_Volume_x_1', 'Lag_Close_x_5', 'Lag_Volume_x_5', 'Lag_Close_x_10', 'Lag_Volume_x_10', 'Day_of_Week', 'Month', 'Quarter', 'W

In [1]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# Load a small chunk of dataset for testing
print("Loading a small chunk of dataset for testing...")
df = pd.read_parquet('final_with_external_features_and_rates.parquet')  # Replace with the correct file path
df = df.head(1000)  # Load only a small chunk for testing
print("Dataset chunk loaded.")
print("Initial columns:", df.columns.tolist())

# Ensure 'Date' column is in datetime format in the main dataset
df['Date'] = pd.to_datetime(df['Date'])

# Function to fetch Yahoo Finance data
def fetch_yahoo_data(symbol, feature_name):
    print(f"Fetching {feature_name} data for {symbol}...")
    try:
        data = yf.download(symbol, start=df['Date'].min(), end=df['Date'].max())
        if not data.empty:
            data = data[['Adj Close']].rename(columns={'Adj Close': feature_name})
            data['Date'] = data.index
            data.reset_index(drop=True, inplace=True)
            data['Date'] = pd.to_datetime(data['Date'])
            print(f"Fetched {len(data)} records for {feature_name}.")
            return data[['Date', feature_name]]
        else:
            print(f"No data found for {symbol}")
            return pd.DataFrame(columns=['Date', feature_name])
    except Exception as e:
        print(f"Failed to fetch {feature_name} data for {symbol}: {e}")
        return pd.DataFrame(columns=['Date', feature_name])

# Function to merge data in batches
def merge_in_batches(df, data_to_merge, feature_name):
    if data_to_merge.empty:
        print(f"Skipping {feature_name} - no data available to merge.")
        return df
    print(f"Merging {feature_name} with the main dataset...")
    df = df.merge(data_to_merge, on='Date', how='left')
    return df

# Fetch external data for testing
print("Fetching external data for testing...")
vix_data = fetch_yahoo_data('^VIX', 'VIX_Close')
crude_oil_data = fetch_yahoo_data('CL=F', 'Crude_Oil_Close')
gold_data = fetch_yahoo_data('GC=F', 'Gold_Close')
usd_inr_data = fetch_yahoo_data('INR=X', 'USD_INR_Close')
sp500_data = fetch_yahoo_data('^GSPC', 'SP500_Close')
nifty50_data = fetch_yahoo_data('^NSEI', 'Nifty50_Close')
ftse100_data = fetch_yahoo_data('^FTSE', 'FTSE100_Close')
dax_data = fetch_yahoo_data('^GDAXI', 'DAX_Close')
nikkei225_data = fetch_yahoo_data('^N225', 'Nikkei225_Close')

# Check if date ranges overlap
print("Main dataset date range:", df['Date'].min(), "to", df['Date'].max())
print("VIX_Close date range:", vix_data['Date'].min(), "to", vix_data['Date'].max())
print("Crude_Oil_Close date range:", crude_oil_data['Date'].min(), "to", crude_oil_data['Date'].max())
print("Gold_Close date range:", gold_data['Date'].min(), "to", gold_data['Date'].max())
print("USD_INR_Close date range:", usd_inr_data['Date'].min(), "to", usd_inr_data['Date'].max())
print("SP500_Close date range:", sp500_data['Date'].min(), "to", sp500_data['Date'].max())
print("Nifty50_Close date range:", nifty50_data['Date'].min(), "to", nifty50_data['Date'].max())
print("FTSE100_Close date range:", ftse100_data['Date'].min(), "to", ftse100_data['Date'].max())
print("DAX_Close date range:", dax_data['Date'].min(), "to", dax_data['Date'].max())
print("Nikkei225_Close date range:", nikkei225_data['Date'].min(), "to", nikkei225_data['Date'].max())

# Merging VIX_Close
df = merge_in_batches(df, vix_data, 'VIX_Close')
print("Columns after merging VIX_Close:", df.columns.tolist())

# Merging Crude Oil data
df = merge_in_batches(df, crude_oil_data, 'Crude_Oil_Close')
print("Columns after merging Crude_Oil_Close:", df.columns.tolist())

# Merging Gold data
df = merge_in_batches(df, gold_data, 'Gold_Close')
print("Columns after merging Gold_Close:", df.columns.tolist())

# Merging USD/INR data
df = merge_in_batches(df, usd_inr_data, 'USD_INR_Close')
print("Columns after merging USD_INR_Close:", df.columns.tolist())

# Merging SP500 data
df = merge_in_batches(df, sp500_data, 'SP500_Close')
print("Columns after merging SP500_Close:", df.columns.tolist())

# Merging Nifty50 data
df = merge_in_batches(df, nifty50_data, 'Nifty50_Close')
print("Columns after merging Nifty50_Close:", df.columns.tolist())

# Merging FTSE 100 data
df = merge_in_batches(df, ftse100_data, 'FTSE100_Close')
print("Columns after merging FTSE100_Close:", df.columns.tolist())

# Merging DAX data
df = merge_in_batches(df, dax_data, 'DAX_Close')
print("Columns after merging DAX_Close:", df.columns.tolist())

# Merging Nikkei 225 data
df = merge_in_batches(df, nikkei225_data, 'Nikkei225_Close')
print("Columns after merging Nikkei225_Close:", df.columns.tolist())

# Check missing values and print a preview
print("\nMissing values after merging:\n", df.isnull().sum())
print("\nPreview of merged data:\n", df.head())

# Proceed or stop based on user input
proceed = input("Do you want to continue with the full dataset? (yes/no): ")
if proceed.lower() == 'yes':
    print("Loading full dataset...")
    full_df = pd.read_parquet('final_with_external_features_and_rates.parquet')
    full_df['Date'] = pd.to_datetime(full_df['Date'])

    # Merging the full dataset with VIX data
    full_df = merge_in_batches(full_df, vix_data, 'VIX_Close')

    # Merging the full dataset with Crude Oil data
    full_df = merge_in_batches(full_df, crude_oil_data, 'Crude_Oil_Close')

    # Merging the full dataset with Gold data
    full_df = merge_in_batches(full_df, gold_data, 'Gold_Close')

    # Merging the full dataset with USD/INR data
    full_df = merge_in_batches(full_df, usd_inr_data, 'USD_INR_Close')

    # Merging the full dataset with SP500 data
    full_df = merge_in_batches(full_df, sp500_data, 'SP500_Close')

    # Merging the full dataset with Nifty50 data
    full_df = merge_in_batches(full_df, nifty50_data, 'Nifty50_Close')

    # Merging the full dataset with FTSE 100 data
    full_df = merge_in_batches(full_df, ftse100_data, 'FTSE100_Close')

    # Merging the full dataset with DAX data
    full_df = merge_in_batches(full_df, dax_data, 'DAX_Close')

    # Merging the full dataset with Nikkei 225 data
    full_df = merge_in_batches(full_df, nikkei225_data, 'Nikkei225_Close')

    # Save the final dataset with external features
    full_df.to_parquet('final_with_external_features_and_rates_full.parquet')
    print("Final dataset with external features saved as 'final_with_external_features_and_rates_full.parquet'.")
else:
    print("Execution stopped as per user request.")


Loading a small chunk of dataset for testing...
Dataset chunk loaded.
Initial columns: ['Date', 'Open_x', 'High_x', 'Low_x', 'Close_x', 'Adj Close_x', 'Volume_x', 'Company_ID', 'VWAP_x', 'MACD_x', 'MACD_Signal_x', 'MACD_Hist_x', 'SMA_20', 'EMA_20', 'WMA_20', 'Upper_Band', 'Middle_Band', 'Lower_Band', 'ATR_14', 'RSI_14', 'Stoch_K', 'Stoch_D', 'OBV', 'SAR', 'CCI_20', 'ROC', 'Price_Change', 'Price_Change_Percent', 'Volume_Change', 'Volume_Change_Percent', 'High_Low_Spread', 'Close_Open_Ratio', 'High_Low_Ratio', 'Price_Range', 'Price_Volume_Product', 'Lag_Close_x_1', 'Lag_Volume_x_1', 'Lag_Close_x_5', 'Lag_Volume_x_5', 'Lag_Close_x_10', 'Lag_Volume_x_10', 'Day_of_Week', 'Month', 'Quarter', 'Week_of_Year', 'Rolling_Std_Close', 'Rolling_Std_Volume', 'Rolling_Mean_Close', 'Rolling_Max_Close', 'Rolling_Min_Close', 'Rolling_Price_Range', 'Z_Score_Close', 'Momentum', 'Price_Acceleration', 'OBV_Change', 'Volume_Ratio', 'Rolling_Return', 'Kurtosis', 'Skewness', 'Sharpe_Ratio', 'Fib_Level_0.236', '

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Fetched 1022 records for VIX_Close.
Fetching Crude_Oil_Close data for CL=F...
Fetched 1022 records for Crude_Oil_Close.
Fetching Gold_Close data for GC=F...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Fetched 1022 records for Gold_Close.
Fetching USD_INR_Close data for INR=X...
Fetched 1057 records for USD_INR_Close.
Fetching SP500_Close data for ^GSPC...


...

Fetched 1024 records for FTSE100_Close.
Fetching DAX_Close data for ^GDAXI...
Fetched 1035 records for DAX_Close.
Fetching Nikkei225_Close data for ^N225...


[*********************100%***********************]  1 of 1 completed


Fetched 993 records for Nikkei225_Close.
Main dataset date range: 2008-10-21 00:00:00 to 2012-11-12 00:00:00
VIX_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
Crude_Oil_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
Gold_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
USD_INR_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
SP500_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
Nifty50_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
FTSE100_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
DAX_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
Nikkei225_Close date range: 2008-10-21 00:00:00 to 2012-11-09 00:00:00
Merging VIX_Close with the main dataset...
Columns after merging VIX_Close: ['Date', 'Open_x', 'High_x', 'Low_x', 'Close_x', 'Adj Close_x', 'Volume_x', 'Company_ID', 'VWAP_x', 'MACD_x', 'MACD_Signal_x', 'MACD_Hist_x', 'SMA_20', 'EMA_20', 'WMA_20', 'Upper_Band', 'Middle_Band', 'L

Do you want to continue with the full dataset? (yes/no): yes
Loading full dataset...
Merging VIX_Close with the main dataset...
Merging Crude_Oil_Close with the main dataset...


KeyboardInterrupt: 

In [2]:
print(f"Original dataset size: {len(df)} rows")
# Check size of each external dataset
print(f"VIX data size: {len(vix_data)} rows")
print(f"US CPI data size: {len(us_inflation_data)} rows")
print(f"US Federal Funds Rate data size: {len(us_interest_rate_data)} rows")


Original dataset size: 6409090 rows
VIX data size: 8476 rows
US CPI data size: 0 rows
US Federal Funds Rate data size: 0 rows


In [3]:
df = df.merge(vix_data, on='Date', how='left')
df = df.merge(us_inflation_data, on='Date', how='left')
df = df.merge(us_interest_rate_data, on='Date', how='left')

KeyboardInterrupt: 

In [3]:
import pandas as pd
temp=pd.read_parquet('final_with_external_features_and_rates_and_inflation.parquet')
temp.head()

,Date,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Company_ID,VWAP_x,MACD_x,...,Market_Close,Market_Returns,Global_Close,Global_Returns,Rolling_Beta,Rolling_Market_Correlation,Rolling_Global_Correlation,INR_USD_Close,Crude_Oil_Close,Gold_Close
0,2008-10-21,11.400,11.475,10.550,10.85,9.477412,177410.0,20MICRONS,10.958334,-1.032321,...,3234.899902,0.035897,955.049988,-0.030800,NaN,NaN,NaN,48.896999,70.889999,766.099976
1,2008-10-22,10.925,11.100,10.000,10.15,8.865965,328698.0,20MICRONS,10.416667,-1.032321,...,3065.149902,-0.052475,896.780029,-0.061012,NaN,NaN,NaN,49.208000,66.750000,733.299988
2,2008-10-23,9.150,10.125,8.150,9.50,8.298196,108936.0,20MICRONS,9.258333,-1.032321,...,2943.149902,-0.039802,908.109985,0.012634,NaN,NaN,NaN,49.584999,67.839996,713.299988
3,2008-10-24,8.625,9.950,8.025,8.25,7.206326,122200.0,20MICRONS,8.741666,-1.032321,...,2584.000000,-0.122029,876.770020,-0.034511,NaN,NaN,NaN,49.755001,64.150002,729.099976
4,2008-10-27,7.625,8.475,6.750,7.30,6.376507,141400.0,20MICRONS,7.508334,-1.032321,...,2524.199951,-0.023142,848.919983,-0.031764,NaN,NaN,NaN,49.720001,63.220001,741.700012


In [1]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# Function to fetch Yahoo Finance data
def fetch_yahoo_data(symbol, feature_name, start_date, end_date):
    print(f"Fetching {feature_name} data for {symbol}...")
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        if not data.empty:
            data = data[['Adj Close']].rename(columns={'Adj Close': feature_name})
            data['Date'] = data.index
            data.reset_index(drop=True, inplace=True)
            data['Date'] = pd.to_datetime(data['Date'])
            print(f"Fetched {len(data)} records for {feature_name}.")
            return data[['Date', feature_name]]
        else:
            print(f"No data found for {symbol}")
            return pd.DataFrame(columns=['Date', feature_name])
    except Exception as e:
        print(f"Failed to fetch {feature_name} data for {symbol}: {e}")
        return pd.DataFrame(columns=['Date', feature_name])

# Function to merge data in batches
def merge_in_batches(df, data_to_merge, feature_name, batch_size=5000):
    if data_to_merge.empty:
        print(f"Skipping {feature_name} - no data available to merge.")
        return df
    print(f"Merging {feature_name} with the main dataset in batches...")
    
    num_batches = len(df) // batch_size + 1
    for i in tqdm(range(num_batches), desc=f"Merging {feature_name}"):
        start = i * batch_size
        end = min((i + 1) * batch_size, len(df))
        df.loc[start:end, :] = df.loc[start:end, :].merge(data_to_merge, on='Date', how='left')
    
    print(f"Columns after merging {feature_name}: {len(df.columns)} total columns.")
    return df

# Function to process external data and merge with main dataset
def process_and_merge(df, symbols, batch_size):
    # Fetch and merge data for each symbol
    for symbol, feature_name in symbols.items():
        external_data = fetch_yahoo_data(symbol, feature_name, df['Date'].min(), df['Date'].max())
        df = merge_in_batches(df, external_data, feature_name, batch_size=batch_size)
    return df

# Load full dataset and process in batches
def process_full_dataset(file_path, symbols, batch_size=5000):
    print("Loading the full dataset...")
    df = pd.read_parquet(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    print(f"Dataset loaded with {len(df)} rows.")
    
    # Merge external data in batches
    df = process_and_merge(df, symbols, batch_size=batch_size)
    
    # Save the processed dataset
    output_file = 'final_with_external_features_and_rates_full.parquet'
    df.to_parquet(output_file, index=False)
    print(f"Final dataset with external features saved as '{output_file}'.")

# Define external data symbols and features
symbols = {
    '^VIX': 'VIX_Close',
    'CL=F': 'Crude_Oil_Close',
    'GC=F': 'Gold_Close',
    'INR=X': 'USD_INR_Close',
    '^GSPC': 'SP500_Close',
    '^NSEI': 'Nifty50_Close',
    '^FTSE': 'FTSE100_Close',
    '^GDAXI': 'DAX_Close',
    '^N225': 'Nikkei225_Close'
}

# Set file path for the main dataset
file_path = 'final_with_external_features_and_rates.parquet'

# Process the dataset with external data
process_full_dataset(file_path, symbols, batch_size=1000)



Loading the full dataset...
Dataset loaded with 6409090 rows.
Fetching VIX_Close data for ^VIX...


[*********************100%***********************]  1 of 1 completed


Fetched 8476 records for VIX_Close.
Merging VIX_Close with the main dataset in batches...


Merging VIX_Close:   7%|████▏                                                       | 454/6410 [04:20<56:53,  1.74it/s]


KeyboardInterrupt: 

In [2]:
import pyarrow.parquet as pq
import pandas as pd
from tqdm import tqdm
import os

# Function to fetch all external data for given symbols and date range
def fetch_all_external_data(symbols, start_date, end_date):
    external_data_dict = {}
    for feature_name, symbol in symbols.items():
        print(f"Fetching {feature_name} data for {symbol}...")
        data = fetch_yahoo_data(symbol, feature_name, start_date, end_date)
        external_data_dict[feature_name] = data
    return external_data_dict

# Function to merge all external data into the main dataset chunk
def merge_all_external_data(df_chunk, external_data_dict):
    for feature_name, data in external_data_dict.items():
        df_chunk = df_chunk.merge(data, on='Date', how='left')
    return df_chunk

# Function to fetch Yahoo Finance data
def fetch_yahoo_data(symbol, feature_name, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        if not data.empty:
            data = data[['Adj Close']].rename(columns={'Adj Close': feature_name})
            data['Date'] = data.index
            data.reset_index(drop=True, inplace=True)
            print(f"Fetched {len(data)} records for {feature_name}.")
            return data[['Date', feature_name]]
        else:
            print(f"No data found for {symbol}")
            return pd.DataFrame(columns=['Date', feature_name])
    except Exception as e:
        print(f"Failed to fetch {feature_name} data for {symbol}: {e}")
        return pd.DataFrame(columns=['Date', feature_name])

# Function to process the dataset in batches and save each batch as a Parquet file
def process_full_dataset_in_batches_parquet(file_path, symbols, batch_size=5000):
    # Open the Parquet file
    parquet_file = pq.ParquetFile(file_path)

    # Read the first and last row to get the start and end dates
    first_row = parquet_file.read_row_group(0, columns=['Date']).to_pandas().iloc[0]
    last_row = parquet_file.read_row_group(parquet_file.num_row_groups - 1, columns=['Date']).to_pandas().iloc[-1]
    
    start_date, end_date = first_row['Date'], last_row['Date']
    print(f"Processing data from {start_date} to {end_date}")
    
    # Fetch all external data for the given date range
    external_data_dict = fetch_all_external_data(symbols, start_date, end_date)
    
    # Process each row group in the Parquet file
    for i in tqdm(range(parquet_file.num_row_groups), desc="Processing Parquet file in batches"):
        df_chunk = parquet_file.read_row_group(i).to_pandas()
        print(f"\nProcessing batch {i + 1} with {len(df_chunk)} rows...")
        
        # Merge all external data for this chunk
        df_chunk = merge_all_external_data(df_chunk, external_data_dict)
        print(f"Columns after merging: {df_chunk.shape[1]} total columns.")

        # Save each processed batch as a separate Parquet file
        output_file = f"processed_batch_{i + 1}.parquet"
        df_chunk.to_parquet(output_file)
        print(f"Batch {i + 1} saved as {output_file}.")

    # Combine all processed batches into a single Parquet file
    combine_batches_to_final(file_path)

# Function to combine all processed batches into a single dataset
def combine_batches_to_final(original_file_path):
    output_files = [f for f in os.listdir() if f.startswith('processed_batch_') and f.endswith('.parquet')]
    combined_df = pd.DataFrame()

    print("Combining processed batches into the final dataset...")
    
    for file in tqdm(output_files, desc="Merging batches"):
        batch_df = pd.read_parquet(file)
        combined_df = pd.concat([combined_df, batch_df], ignore_index=True)
    
    # Save the combined final dataset
    final_output_path = original_file_path.replace('.parquet', '_final_with_external_features.parquet')
    combined_df.to_parquet(final_output_path)
    print(f"Final dataset with external features saved as {final_output_path}")

# Define the symbols to fetch data for
symbols = {
    'VIX': '^VIX',
    'Crude_Oil': 'CL=F',
    'Gold': 'GC=F',
    'USD_INR': 'INR=X',
    'SP500': '^GSPC',
    'Nifty50': '^NSEI',
    'FTSE100': '^FTSE',
    'DAX': '^GDAXI',
    'Nikkei225': '^N225'
}

file_path = 'final_with_external_features_and_rates.parquet'

# Process the dataset in chunks and merge external data
process_full_dataset_in_batches_parquet(file_path, symbols, batch_size=5000)

print("Final dataset with external features saved in batches.")


Processing data from 2008-10-21 00:00:00 to 2024-09-11 00:00:00
Fetching VIX data for ^VIX...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Fetched 3998 records for VIX.
Fetching Crude_Oil data for CL=F...
Fetched 3996 records for Crude_Oil.
Fetching Gold data for GC=F...


[*********************100%***********************]  1 of 1 completed


Fetched 3995 records for Gold.
Fetching USD_INR data for INR=X...


...

Batch 6 saved as processed_batch_6.parquet.

Processing batch 7 with 117634 rows...
Columns after merging: 91 total columns.


Processing Parquet file in batches: 100%|████████████████████████████████████████████████| 7/7 [01:17<00:00, 11.11s/it]


Batch 7 saved as processed_batch_7.parquet.
Combining processed batches into the final dataset...


Merging batches: 100%|███████████████████████████████████████████████████████████████████| 7/7 [00:54<00:00,  7.74s/it]


Final dataset with external features saved as final_with_external_features_and_rates_final_with_external_features.parquet
Final dataset with external features saved in batches.


In [4]:
import pandas as pd

def check_final_dataset(file_path):
    # Load the final combined dataset
    print(f"Loading the final dataset from {file_path}...")
    df = pd.read_parquet(file_path)

    # Check the columns list
    print("\nList of Columns in the Dataset:")
    print(df.columns.tolist())

    # Check the number of rows and columns
    print(f"\nThe dataset has {df.shape[0]} rows and {df.shape[1]} columns.")

    # Check for missing values
    print("\nMissing values in the dataset:")
    missing_values = df.isnull().sum()
    print(missing_values[missing_values > 0])

    # Preview the first few rows of the dataset
    print("\nPreview of the first few rows of the dataset:")
    print(df.head())

# Define the path to the final dataset formed after merging batches
final_file_path = 'final_with_external_features_and_rates_final_with_external_features.parquet'

# Run the check
check_final_dataset(final_file_path)


Loading the final dataset from final_with_external_features_and_rates_final_with_external_features.parquet...

List of Columns in the Dataset:
['Date', 'Open_x', 'High_x', 'Low_x', 'Close_x', 'Adj Close_x', 'Volume_x', 'Company_ID', 'VWAP_x', 'MACD_x', 'MACD_Signal_x', 'MACD_Hist_x', 'SMA_20', 'EMA_20', 'WMA_20', 'Upper_Band', 'Middle_Band', 'Lower_Band', 'ATR_14', 'RSI_14', 'Stoch_K', 'Stoch_D', 'OBV', 'SAR', 'CCI_20', 'ROC', 'Price_Change', 'Price_Change_Percent', 'Volume_Change', 'Volume_Change_Percent', 'High_Low_Spread', 'Close_Open_Ratio', 'High_Low_Ratio', 'Price_Range', 'Price_Volume_Product', 'Lag_Close_x_1', 'Lag_Volume_x_1', 'Lag_Close_x_5', 'Lag_Volume_x_5', 'Lag_Close_x_10', 'Lag_Volume_x_10', 'Day_of_Week', 'Month', 'Quarter', 'Week_of_Year', 'Rolling_Std_Close', 'Rolling_Std_Volume', 'Rolling_Mean_Close', 'Rolling_Max_Close', 'Rolling_Min_Close', 'Rolling_Price_Range', 'Z_Score_Close', 'Momentum', 'Price_Acceleration', 'OBV_Change', 'Volume_Ratio', 'Rolling_Return', 'Kur

In [5]:
import pandas as pd

def check_missing_values(file_path):
    print(f"Loading the dataset from {file_path}...")
    df = pd.read_parquet(file_path)
    
    print("\nMissing values in the dataset:")
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0]
    
    print(missing_values)
    return missing_values

# Check missing values
final_file_path = 'final_with_all_features.parquet'
check_missing_values(final_file_path)


Loading the dataset from final_with_all_features.parquet...

Missing values in the dataset:
COMPANY                          6650
Rolling_Beta                       62
Rolling_Market_Correlation         62
Rolling_Global_Correlation         62
VIX                           1330237
Crude_Oil                     1332743
Gold                          1334145
USD_INR                       1181160
SP500                         1330237
Nifty50                       1198061
FTSE100                       1312526
DAX                           1270631
Nikkei225                     1508202
dtype: int64


COMPANY                          6650
Rolling_Beta                       62
Rolling_Market_Correlation         62
Rolling_Global_Correlation         62
VIX                           1330237
Crude_Oil                     1332743
Gold                          1334145
USD_INR                       1181160
SP500                         1330237
Nifty50                       1198061
FTSE100                       1312526
DAX                           1270631
Nikkei225                     1508202
dtype: int64

In [4]:
len(temp)

6409090

In [7]:
import pandas as pd
import os

def handle_missing_values_by_company(file_path, batch_size=1000):
    parquet_file = pd.read_parquet(file_path, engine='pyarrow')
    companies = parquet_file['Company_ID'].unique()
    
    print("Processing missing values by company in batches...")
    
    processed_batches = []
    
    for i in range(0, len(companies), batch_size):
        # Process a batch of companies
        batch_companies = companies[i:i + batch_size]
        
        print(f"\nProcessing companies batch {i//batch_size + 1} with companies: {batch_companies}")
        
        # Filter dataset for the current batch of companies
        df_batch = parquet_file[parquet_file['Company_ID'].isin(batch_companies)]
        
        # Handle missing values for numerical columns
        numerical_cols = df_batch.select_dtypes(include=['float64', 'int64']).columns
        df_batch[numerical_cols] = df_batch[numerical_cols].fillna(method='ffill').fillna(method='bfill')
        
        # Fill remaining missing numerical data with the median
        for col in numerical_cols:
            df_batch[col].fillna(df_batch[col].median(), inplace=True)

        # Handle missing values for categorical columns
        categorical_cols = df_batch.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            df_batch[col].fillna(df_batch[col].mode()[0], inplace=True)
        
        # Save the processed batch
        batch_file_path = f"processed_company_batch_{i//batch_size + 1}.parquet"
        df_batch.to_parquet(batch_file_path)
        processed_batches.append(batch_file_path)
        
        print(f"Batch {i//batch_size + 1} processed and saved as {batch_file_path}")

    # After processing all batches, merge them back into one final dataset
    print("\nMerging processed batches into a final dataset...")
    merged_df = pd.concat([pd.read_parquet(batch_file) for batch_file in processed_batches], ignore_index=True)
    
    # Save the merged file as final_cleaned.parquet
    final_file_path = "final_cleaned.parquet"
    merged_df.to_parquet(final_file_path)
    
    print(f"\nFinal dataset with merged batches saved as {final_file_path}")
    
    # Clean up the intermediate files after merging
    for batch_file in processed_batches:
        os.remove(batch_file)
        print(f"Intermediate batch file {batch_file} removed.")

# Run the function to handle missing values and merge batches for the entire dataset
handle_missing_values_by_company('final_with_all_features.parquet', batch_size=100)


Processing missing values by company in batches...

Processing companies batch 1 with companies: ['20MICRONS' '21STCENMGM' '360ONE' '3IINFOLTD' '3MINDIA' '3PLAND' '5PAISA'
 '63MOONS' 'A2ZINFRA' 'AAATECH' 'AADHARHFC' 'AAKASH' 'AAREYDRUGS' 'AARON'
 'AARTECH' 'AARTIDRUGS' 'AARTIIND' 'AARTIPHARM' 'AARTISURF' 'AARVEEDEN'
 'AARVI' 'AAVAS' 'ABAN' 'ABB' 'ABBOTINDIA' 'ABCAPITAL' 'ABDL' 'ABFRL'
 'ABMINTLLTD' 'ABSLAMC' 'ACC' 'ACCELYA' 'ACCURACY' 'ACE' 'ACEINTEG' 'ACI'
 'ACL' 'ACLGATI' 'ADANIENSOL' 'ADANIENT' 'ADANIGREEN' 'ADANIPORTS'
 'ADANIPOWER' 'ADFFOODS' 'ADL' 'ADORWELD' 'ADROITINFO' 'ADSL' 'ADVANIHOTR'
 'ADVENZYMES' 'AEGISLOG' 'AEROFLEX' 'AETHER' 'AFFLE' 'AFIL' 'AGARIND'
 'AGI' 'AGIIL' 'AGRITECH' 'AGROPHOS' 'AGSTRA' 'AHL' 'AHLADA' 'AHLEAST'
 'AHLUCONT' 'AIAENG' 'AIIL' 'AIRAN' 'AIROLAM' 'AJANTPHARM' 'AJMERA'
 'AJOONI' 'AKASH' 'AKG' 'AKI' 'AKSHAR' 'AKSHARCHEM' 'AKSHOPTFBR' 'AKUMS'
 'AKZOINDIA' 'ALANKIT' 'ALBERTDAVD' 'ALEMBICLTD' 'ALICON' 'ALKALI' 'ALKEM'
 'ALKYLAMINE' 'ALLCARGO' 'ALLSEC' 'ALMO

C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch[numerical_cols] = df_batch[numerical_cols].fillna(method='ffill').fillna(method='bfill')
C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch[col].fillna(df_batch[col].median(), inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documen

Batch 1 processed and saved as processed_company_batch_1.parquet

Processing companies batch 2 with companies: ['AMJLAND' 'AMNPLST' 'AMRUTANJAN' 'ANANDRATHI' 'ANANTRAJ' 'ANDHRAPAP'
 'ANDHRSUGAR' 'ANGELONE' 'ANIKINDS' 'ANMOL' 'ANSALAPI' 'ANTGRAPHIC' 'ANUP'
 'ANURAS' 'APARINDS' 'APCL' 'APCOTEXIND' 'APEX' 'APLAPOLLO' 'APLLTD'
 'APOLLO' 'APOLLOHOSP' 'APOLLOPIPE' 'APOLLOTYRE' 'APOLSINHOT' 'APTECHT'
 'APTUS' 'ARCHIDPLY' 'ARCHIES' 'ARE&M' 'ARENTERP' 'ARIES' 'ARIHANTCAP'
 'ARIHANTSUP' 'ARMANFIN' 'AROGRANITE' 'ARROWGREEN' 'ARSSINFRA'
 'ARTEMISMED' 'ARTNIRMAN' 'ARVEE' 'ARVIND' 'ARVINDFASN' 'ARVSMART'
 'ASAHIINDIA' 'ASAHISONG' 'ASAL' 'ASALCBR' 'ASHAPURMIN' 'ASHIANA'
 'ASHIMASYN' 'ASHOKA' 'ASHOKAMET' 'ASHOKLEY' 'ASIANENE' 'ASIANHOTNR'
 'ASIANPAINT' 'ASIANTILES' 'ASKAUTOLTD' 'ASMS' 'ASPINWALL' 'ASTEC'
 'ASTERDM' 'ASTRAL' 'ASTRAMICRO' 'ASTRAZEN' 'ASTRON' 'ATALREAL' 'ATAM'
 'ATFL' 'ATGL' 'ATL' 'ATLANTAA' 'ATUL' 'ATULAUTO' 'AUBANK' 'AURIONPRO'
 'AUROPHARMA' 'AURUM' 'AUSOMENT' 'AUTOAXLES' 'AUTOIND' 'AV

C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch[numerical_cols] = df_batch[numerical_cols].fillna(method='ffill').fillna(method='bfill')
C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch[col].fillna(df_batch[col].median(), inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documen

Batch 2 processed and saved as processed_company_batch_2.parquet

Processing companies batch 3 with companies: ['BAIDFIN' 'BAJAJ-AUTO' 'BAJAJCON' 'BAJAJELEC' 'BAJAJFINSV' 'BAJAJHCARE'
 'BAJAJHIND' 'BAJAJHLDNG' 'BAJEL' 'BAJFINANCE' 'BALAJITELE' 'BALAMINES'
 'BALAXI' 'BALKRISHNA' 'BALKRISIND' 'BALMLAWRIE' 'BALPHARMA' 'BALRAMCHIN'
 'BALUFORGE' 'BANARBEADS' 'BANARISUG' 'BANCOINDIA' 'BANDHANBNK' 'BANG'
 'BANKA' 'BANKBARODA' 'BANKINDIA' 'BANSALWIRE' 'BANSWRAS' 'BARBEQUE'
 'BASF' 'BASML' 'BATAINDIA' 'BAYERCROP' 'BBL' 'BBOX' 'BBTC' 'BBTCL'
 'BCLIND' 'BCONCEPTS' 'BDL' 'BEARDSELL' 'BECTORFOOD' 'BEDMUTHA' 'BEL'
 'BEML' 'BEPL' 'BERGEPAINT' 'BESTAGRO' 'BFINVEST' 'BFUTILITIE' 'BGRENERGY'
 'BHAGCHEM' 'BHAGERIA' 'BHAGYANGR' 'BHANDARI' 'BHARATFORG' 'BHARATGEAR'
 'BHARATRAS' 'BHARATWIRE' 'BHARTIARTL' 'BHARTIHEXA' 'BHEL' 'BIGBLOC'
 'BIKAJI' 'BIL' 'BINANIIND' 'BIOCON' 'BIOFILCHEM' 'BIRLACABLE'
 'BIRLACORPN' 'BIRLAMONEY' 'BLAL' 'BLBLIMITED' 'BLISSGVS' 'BLKASHYAP'
 'BLS' 'BLSE' 'BLUECHIP' 'BLUEDART' 'BLUEJE

...

Batch 18 processed and saved as processed_company_batch_18.parquet

Processing companies batch 19 with companies: ['TIPSFILMS' 'TIPSINDLTD' 'TIRUMALCHM' 'TIRUPATIFL' 'TITAGARH' 'TITAN'
 'TMB' 'TNPETRO' 'TNPL' 'TNTELE' 'TOKYOPLAST' 'TORNTPHARM' 'TORNTPOWER'
 'TOTAL' 'TOUCHWOOD' 'TPHQ' 'TPLPLASTEH' 'TRACXN' 'TREEHOUSE' 'TREJHARA'
 'TREL' 'TRENT' 'TRF' 'TRIDENT' 'TRIGYN' 'TRITURBINE' 'TRIVENI' 'TRU'
 'TTKHLTCARE' 'TTKPRESTIG' 'TTL' 'TTML' 'TV18BRDCST' 'TVSELECT' 'TVSHLTD'
 'TVSMOTOR' 'TVSSCS' 'TVSSRICHAK' 'TVTODAY' 'TVVISION' 'UBL' 'UCAL'
 'UCOBANK' 'UDAICEMENT' 'UDS' 'UFLEX' 'UFO' 'UGARSUGAR' 'UGROCAP'
 'UJJIVANSFB' 'ULTRACEMCO' 'UMAEXPORTS' 'UMANGDAIRY' 'UMESLTD'
 'UNICHEMLAB' 'UNIDT' 'UNIECOM' 'UNIENTER' 'UNIINFO' 'UNIONBANK'
 'UNIPARTS' 'UNITDSPR' 'UNITECH' 'UNITEDPOLY' 'UNITEDTEA' 'UNIVASTU'
 'UNIVCABLES' 'UNIVPHOTO' 'UNOMINDA' 'UPL' 'URAVI' 'URJA' 'USHAMART' 'USK'
 'UTIAMC' 'UTKARSHBNK' 'UTTAMSUGAR' 'V2RETAIL' 'VADILALIND' 'VAIBHAVGBL'
 'VAISHALI' 'VAKRANGEE' 'VALIANTLAB' 'VALIANTOR

C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch[numerical_cols] = df_batch[numerical_cols].fillna(method='ffill').fillna(method='bfill')
C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch[col].fillna(df_batch[col].median(), inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documen

Batch 19 processed and saved as processed_company_batch_19.parquet

Processing companies batch 20 with companies: ['VHL' 'VHLTD' 'VIDHIING' 'VIJAYA' 'VIJIFIN' 'VIKASECO' 'VIKASLIFE'
 'VIMTALABS' 'VINATIORGA' 'VINDHYATEL' 'VINEETLAB' 'VINNY' 'VINYLINDIA'
 'VIPCLOTHNG' 'VIPIND' 'VIPULLTD' 'VIRINCHI' 'VISAKAIND' 'VISHNU'
 'VISHWARAJ' 'VIVIDHA' 'VLEGOV' 'VLSFINANCE' 'VMART' 'VOLTAMP' 'VOLTAS'
 'VPRPL' 'VRAJ' 'VRLLOG' 'VSSL' 'VSTIND' 'VSTL' 'VSTTILLERS' 'VTL' 'WABAG'
 'WALCHANNAG' 'WANBURY' 'WEALTH' 'WEBELSOLAR' 'WEIZMANIND' 'WEL' 'WELCORP'
 'WELENT' 'WELINV' 'WELSPUNLIV' 'WENDT' 'WESTLIFE' 'WEWIN' 'WHEELS'
 'WHIRLPOOL' 'WILLAMAGOR' 'WINDLAS' 'WINDMACHIN' 'WINSOME' 'WIPL' 'WIPRO'
 'WOCKPHARMA' 'WONDERLA' 'WORTH' 'WSI' 'WSTCSTPAPR' 'XCHANGING' 'XELPMOC'
 'XPROINDIA' 'YAARI' 'YASHO' 'YATHARTH' 'YATRA' 'YESBANK' 'YUKEN' 'ZAGGLE'
 'ZEEL' 'ZEELEARN' 'ZEEMEDIA' 'ZENITHEXPO' 'ZENITHSTL' 'ZENSARTECH'
 'ZENTEC' 'ZFCVINDIA' 'ZIMLAB' 'ZODIAC' 'ZODIACLOTH' 'ZOMATO' 'ZOTA'
 'ZUARI' 'ZUARIIND' 'ZYDUSLIFE

C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch[numerical_cols] = df_batch[numerical_cols].fillna(method='ffill').fillna(method='bfill')
C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch[col].fillna(df_batch[col].median(), inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_6500\2457953110.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documen

Batch 20 processed and saved as processed_company_batch_20.parquet

Merging processed batches into a final dataset...

Final dataset with merged batches saved as final_cleaned.parquet
Intermediate batch file processed_company_batch_1.parquet removed.
Intermediate batch file processed_company_batch_2.parquet removed.
Intermediate batch file processed_company_batch_3.parquet removed.
Intermediate batch file processed_company_batch_4.parquet removed.
Intermediate batch file processed_company_batch_5.parquet removed.
Intermediate batch file processed_company_batch_6.parquet removed.
Intermediate batch file processed_company_batch_7.parquet removed.
Intermediate batch file processed_company_batch_8.parquet removed.
Intermediate batch file processed_company_batch_9.parquet removed.
Intermediate batch file processed_company_batch_10.parquet removed.
Intermediate batch file processed_company_batch_11.parquet removed.
Intermediate batch file processed_company_batch_12.parquet removed.
Intermedi

In [1]:
import pandas as pd
import os

def handle_missing_values_by_company(file_path, batch_size=1000):
    parquet_file = pd.read_parquet(file_path, engine='pyarrow')
    companies = parquet_file['Company_ID'].unique()
    
    print("Processing missing values by company in batches...")
    
    processed_batches = []
    
    for i in range(0, len(companies), batch_size):
        # Process a batch of companies
        batch_companies = companies[i:i + batch_size]
        
        print(f"\nProcessing companies batch {i//batch_size + 1} with companies: {batch_companies}")
        
        # Filter dataset for the current batch of companies
        df_batch = parquet_file[parquet_file['Company_ID'].isin(batch_companies)].copy()

        # Handle missing values for numerical columns
        numerical_cols = df_batch.select_dtypes(include=['float64', 'int64']).columns
        
        # Use .loc for explicit assignment and avoid the SettingWithCopyWarning
        df_batch.loc[:, numerical_cols] = df_batch[numerical_cols].fillna(method='ffill').fillna(method='bfill')

        # Fill remaining missing numerical data with the median using .loc
        for col in numerical_cols:
            df_batch.loc[:, col].fillna(df_batch[col].median(), inplace=True)

        # Handle missing values for categorical columns
        categorical_cols = df_batch.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            df_batch.loc[:, col].fillna(df_batch[col].mode()[0], inplace=True)
        
        # Save the processed batch
        batch_file_path = f"processed_company_batch_{i//batch_size + 1}.parquet"
        df_batch.to_parquet(batch_file_path)
        processed_batches.append(batch_file_path)
        
        print(f"Batch {i//batch_size + 1} processed and saved as {batch_file_path}")

    # After processing all batches, merge them back into one final dataset
    print("\nMerging processed batches into a final dataset...")
    merged_df = pd.concat([pd.read_parquet(batch_file) for batch_file in processed_batches], ignore_index=True)
    
    # Save the merged file as final_cleaned.parquet
    final_file_path = "final_cleaned.parquet"
    merged_df.to_parquet(final_file_path)
    
    print(f"\nFinal dataset with merged batches saved as {final_file_path}")
    
    # Clean up the intermediate files after merging
    for batch_file in processed_batches:
        os.remove(batch_file)
        print(f"Intermediate batch file {batch_file} removed.")

# Run the function to handle missing values and merge batches for the entire dataset
handle_missing_values_by_company('final_with_all_features.parquet', batch_size=100)


Processing missing values by company in batches...

Processing companies batch 1 with companies: ['20MICRONS' '21STCENMGM' '360ONE' '3IINFOLTD' '3MINDIA' '3PLAND' '5PAISA'
 '63MOONS' 'A2ZINFRA' 'AAATECH' 'AADHARHFC' 'AAKASH' 'AAREYDRUGS' 'AARON'
 'AARTECH' 'AARTIDRUGS' 'AARTIIND' 'AARTIPHARM' 'AARTISURF' 'AARVEEDEN'
 'AARVI' 'AAVAS' 'ABAN' 'ABB' 'ABBOTINDIA' 'ABCAPITAL' 'ABDL' 'ABFRL'
 'ABMINTLLTD' 'ABSLAMC' 'ACC' 'ACCELYA' 'ACCURACY' 'ACE' 'ACEINTEG' 'ACI'
 'ACL' 'ACLGATI' 'ADANIENSOL' 'ADANIENT' 'ADANIGREEN' 'ADANIPORTS'
 'ADANIPOWER' 'ADFFOODS' 'ADL' 'ADORWELD' 'ADROITINFO' 'ADSL' 'ADVANIHOTR'
 'ADVENZYMES' 'AEGISLOG' 'AEROFLEX' 'AETHER' 'AFFLE' 'AFIL' 'AGARIND'
 'AGI' 'AGIIL' 'AGRITECH' 'AGROPHOS' 'AGSTRA' 'AHL' 'AHLADA' 'AHLEAST'
 'AHLUCONT' 'AIAENG' 'AIIL' 'AIRAN' 'AIROLAM' 'AJANTPHARM' 'AJMERA'
 'AJOONI' 'AKASH' 'AKG' 'AKI' 'AKSHAR' 'AKSHARCHEM' 'AKSHOPTFBR' 'AKUMS'
 'AKZOINDIA' 'ALANKIT' 'ALBERTDAVD' 'ALEMBICLTD' 'ALICON' 'ALKALI' 'ALKEM'
 'ALKYLAMINE' 'ALLCARGO' 'ALLSEC' 'ALMO

Batch 8 processed and saved as processed_company_batch_8.parquet

Processing companies batch 9 with companies: ['INDIGO' 'INDIGOPNTS' 'INDNIPPON' 'INDOAMIN' 'INDOBORAX' 'INDOCO'
 'INDORAMA' 'INDOSTAR' 'INDOTECH' 'INDOTHAI' 'INDOWIND' 'INDRAMEDCO'
 'INDSWFTLAB' 'INDSWFTLTD' 'INDTERRAIN' 'INDUSINDBK' 'INDUSTOWER'
 'INFIBEAM' 'INFOBEAN' 'INFOMEDIA' 'INFY' 'INGERRAND' 'INNOVACAP'
 'INOXGREEN' 'INOXINDIA' 'INOXWIND' 'INSECTICID' 'INSPIRISYS' 'INTELLECT'
 'INTENTECH' 'INTLCONV' 'INVENTURE' 'IOB' 'IOC' 'IOLCP' 'IONEXCHANG'
 'IPCALAB' 'IPL' 'IRB' 'IRCON' 'IRCTC' 'IREDA' 'IRFC' 'IRIS' 'IRISDOREME'
 'IRMENERGY' 'ISEC' 'ISFT' 'ISGEC' 'ITC' 'ITDC' 'ITDCEM' 'ITI' 'IVC' 'IVP'
 'IWEL' 'IXIGO' 'IZMO' 'J&KBANK' 'JAGRAN' 'JAGSNPHARM' 'JAIBALAJI'
 'JAICORPLTD' 'JAIPURKURT' 'JAMNAAUTO' 'JASH' 'JAYAGROGN' 'JAYBARMARU'
 'JAYNECOIND' 'JAYSREETEA' 'JBCHEPHARM' 'JBMA' 'JCHAC' 'JETAIRWAYS'
 'JETFREIGHT' 'JGCHEM' 'JHS' 'JINDALPHOT' 'JINDALPOLY' 'JINDALSAW'
 'JINDALSTEL' 'JINDRILL' 'JINDWORLD' 'JIOFIN' 'JISLDVREQ

Batch 16 processed and saved as processed_company_batch_16.parquet

Processing companies batch 17 with companies: ['SHRENIK' 'SHREYANIND' 'SHREYAS' 'SHRIPISTON' 'SHRIRAMFIN' 'SHRIRAMPPS'
 'SHYAMCENT' 'SHYAMMETL' 'SHYAMTEL' 'SIEMENS' 'SIGACHI' 'SIGIND' 'SIGMA'
 'SIGNATURE' 'SIGNPOST' 'SIKKO' 'SIL' 'SILGO' 'SILINV' 'SILLYMONKS'
 'SILVERTUC' 'SIMBHALS' 'SIMPLEXINF' 'SINCLAIR' 'SINDHUTRAD' 'SINTERCOM'
 'SIRCA' 'SIS' 'SITINET' 'SIYSIL' 'SJS' 'SJVN' 'SKFINDIA' 'SKIPPER'
 'SKMEGGPROD' 'SKYGOLD' 'SMARTLINK' 'SMCGLOBAL' 'SMLISUZU' 'SMLT'
 'SMSLIFE' 'SMSPHARMA' 'SNOWMAN' 'SOBHA' 'SOFTTECH' 'SOLARA' 'SOLARINDS'
 'SOMANYCERA' 'SOMATEX' 'SOMICONVEY' 'SONACOMS' 'SONAMLTD' 'SONATSOFTW'
 'SOTL' 'SOUTHBANK' 'SOUTHWEST' 'SPAL' 'SPANDANA' 'SPARC' 'SPCENET'
 'SPECIALITY' 'SPENCERS' 'SPIC' 'SPLIL' 'SPLPETRO' 'SPMLINFRA' 'SPORTKING'
 'SREEL' 'SRF' 'SRGHFL' 'SRHHYPOLTD' 'SRM' 'SRPL' 'SSWL' 'STANLEY' 'STAR'
 'STARCEMENT' 'STARHEALTH' 'STARPAPER' 'STARTECK' 'STCINDIA' 'STEELCAS'
 'STEELCITY' 'STEELXIND' 'STEL'

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import pyarrow.parquet as pq

# Function to remove outliers for a batch
def process_batch(df_batch, confirm_removal=False):
    # Z-Score outlier removal for selected columns
    def remove_outliers_zscore(df, columns, threshold=3):
        outliers = pd.DataFrame()
        for col in columns:
            z_scores = np.abs(stats.zscore(df[col]))
            outliers[col] = z_scores >= threshold
        return df[~outliers.any(axis=1)], outliers.sum().sum()

    # IQR outlier removal for financial ratios
    def remove_outliers_iqr(df, columns):
        outliers = pd.DataFrame()
        for col in columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers[col] = (df[col] < lower_bound) | (df[col] > upper_bound)
        return df[~outliers.any(axis=1)], outliers.sum().sum()

    # Rolling statistics outlier removal for time-series
    def remove_outliers_rolling(df, column, window=20, threshold=3):
        rolling_mean = df[column].rolling(window=window).mean()
        rolling_std = df[column].rolling(window=window).std()
        outliers = np.abs(df[column] - rolling_mean) > threshold * rolling_std
        return df[~outliers], outliers.sum()

    # Z-Score outlier removal for selected columns
    zscore_columns = ['Close_x', 'Volume_x', 'VWAP_x', 'RSI_14', 'MACD_x', 'Price_Change_Percent']
    df_cleaned, zscore_outliers = remove_outliers_zscore(df_batch, zscore_columns)

    # IQR outlier removal for financial ratios
    iqr_columns = ['Close_Open_Ratio', 'High_Low_Ratio', 'Price_Range', 'Volume_Ratio']
    df_cleaned, iqr_outliers = remove_outliers_iqr(df_cleaned, iqr_columns)

    # Rolling statistics outlier removal for time-series
    df_cleaned, rolling_outliers = remove_outliers_rolling(df_cleaned, 'Close_x', window=20, threshold=3)

    # Display outliers detected
    total_outliers = zscore_outliers + iqr_outliers + rolling_outliers
    print(f"Detected {total_outliers} outliers in this batch.")

    # Ask for user confirmation to remove outliers
    if confirm_removal:
        remove_outliers = input(f"Do you want to remove {total_outliers} outliers from this batch? (y/n): ").lower() == 'y'
        if not remove_outliers:
            return df_batch  # Return original data if outliers are not removed

    return df_cleaned

# Function to process the entire dataset in chunks (without parallel processing)
def process_entire_dataset_in_batches(file_path, chunksize=50000, confirm_removal=True):
    # Initialize an empty list to store the cleaned data chunks
    cleaned_data_chunks = []
    rows_processed = 0

    # Load the dataset metadata
    parquet_file = pq.ParquetFile(file_path)
    total_rows = parquet_file.metadata.num_rows

    # Process the file in chunks
    try:
        for batch in tqdm(parquet_file.iter_batches(batch_size=chunksize), desc="Processing Chunks", total=total_rows // chunksize + 1):
            df_chunk = pd.DataFrame(batch.to_pandas())
            rows_processed += len(df_chunk)

            # Process each batch
            cleaned_chunk = process_batch(df_chunk, confirm_removal)
            cleaned_data_chunks.append(cleaned_chunk)

            tqdm.write(f"Processed {rows_processed}/{total_rows} rows")

        # Concatenate the cleaned chunks into a single DataFrame
        cleaned_df = pd.concat(cleaned_data_chunks, ignore_index=True)

        # Save the cleaned data
        output_file = 'final_outliers_2y.parquet'
        cleaned_df.to_parquet(output_file)
        print(f"Cleaned data saved to {output_file}")

        return cleaned_df
    except Exception as e:
        print(f"Error in processing dataset: {e}")

# Example usage: Process the entire dataset
file_path = 'final_cleaned.parquet'
cleaned_df = process_entire_dataset_in_batches(file_path, confirm_removal=True)


Processing Chunks:   0%|                                                                       | 0/129 [00:00<?, ?it/s]

Detected 15298 outliers in this batch.
Do you want to remove 15298 outliers from this batch? (y/n): y


Processing Chunks:   1%|▍                                                              | 1/129 [00:09<20:23,  9.56s/it]

Processed 50000/6409090 rows
Detected 15092 outliers in this batch.
Do you want to remove 15092 outliers from this batch? (y/n): y


Processing Chunks:   2%|▉                                                              | 2/129 [00:31<35:31, 16.78s/it]

...

Processed 6350000/6409090 rows
Detected 17026 outliers in this batch.
Do you want to remove 17026 outliers from this batch? (y/n): y


Processing Chunks:  99%|████████████████████████████████████████████████████████████▌| 128/129 [03:48<00:01,  1.04s/it]

Processed 6400000/6409090 rows
Detected 2248 outliers in this batch.
Do you want to remove 2248 outliers from this batch? (y/n): y


Processing Chunks: 100%|█████████████████████████████████████████████████████████████| 129/129 [03:51<00:00,  1.80s/it]


Processed 6409090/6409090 rows
Cleaned data saved to final_outliers_2.parquet
